In [53]:
import pandas as pd
import numpy as np
import pickle

In [46]:
## this is for loc_1-3 folders, for other location folders refer to lower cells

numLoc = 3
numVec = 24
numScan = 5

vectors = []
## for each location file i.e. range(1,numLoc+1)
for i in range(1,numLoc+1):
    locPath = "loc_" + str(i) + "/"
    db = 1
    ## for each vec file i.e. range(1,numVec+1)
    for j in range(1,numVec+1):
        vecPath = "vec_" + str(j) + "/"
        minRan = db
        maxRan = db+numScan
        for k in range(minRan,maxRan):
            ## print(db)
            dbPath = "db%s.csv" % k
            db+=1
            path = locPath + vecPath + dbPath
            ## "file_{}.dat".format(i)
            data = pd.read_csv(path)
        
            new = data[['MAC Address','RSSI']]
            new = new.set_index('MAC Address')
            if j == 1:
                if k == 1:
                    ## using top 6 initial signal strengths (can be changed for specific case)
                    ## these 6 will refer to 6 highest potential APs
                    ref = new[0:6]
                    reflist = ref.index.values.tolist()
                    # pickle the list for creating prdictions vectors later
                    with open('reflist.pkl', 'wb') as f:
                        pickle.dump(reflist, f)
                    ## print(reflist)
                    base = pd.DataFrame(index=ref.index)
                    base.to_pickle(refbase.pkl)
            vecdf = base
            new = new['RSSI'].to_dict()
            applist = []
            for h in reflist:
                toapp = 0
                for x,y in new.items():
                    if x == h:
                        toapp = y
                applist.append(toapp)
            add = np.asarray(applist)
            vecdf[('RSSI%d' % (db - minRan))] = add
        ## vecdf now refers to all rssi values sorted by scan and AP(MAC Address) for 1 vector
        ## We must now transpose each df, find the average RSSI and append the resulting vector
        ## to our final data df, then export as a csv
        vecdf = vecdf.T
        ## replace all zero values with nan
        vecdf.replace(0, np.nan, inplace= True)
        ## print(vecdf.head())
        ## create the vector which will be fed to NN
        vec = []
    
        for column in vecdf:
            avg = vecdf[column].mean()
            vec.append(avg)
        vec.append(i-1)
        vectors.append(vec)
        print(vec)
print(vectors)
col = reflist
col.append("loc")
allvec=pd.DataFrame(vectors,columns=col)
print(allvec)
allvec.to_csv('trainingdata.csv', index=False)

[-43.666666666666664, -47.4, -50.0, -53.333333333333336, -55.2, -57.0, 0]
[-43.5, -46.25, -49.6, -54.0, -54.8, -56.6, 0]
[-43.666666666666664, -45.25, -50.2, -54.5, -55.0, -55.8, 0]
[-43.5, -47.0, -52.6, -53.75, -54.8, -57.0, 0]
[-44.2, -46.25, -50.0, -53.2, -54.4, -55.6, 0]
[-45.666666666666664, -46.2, -50.6, -53.0, -54.2, -56.0, 0]
[-43.8, -46.6, -49.6, -53.0, -55.2, -55.6, 0]
[-45.0, -46.8, -49.0, -52.4, -55.0, -55.8, 0]
[-45.4, -46.8, -49.0, -53.0, -55.0, -56.0, 0]
[-44.6, -45.4, -49.0, -53.0, -55.0, -56.2, 0]
[-43.75, -46.0, -49.0, -52.333333333333336, -55.0, -55.8, 0]
[-43.6, -46.5, -50.8, -54.6, -55.6, -57.2, 0]
[-43.25, -46.8, -50.75, -55.0, -54.6, -59.0, 0]
[-42.666666666666664, -46.0, -50.8, -55.25, -54.8, -56.2, 0]
[-41.5, -46.8, -51.0, -54.0, -55.0, -56.6, 0]
[-42.333333333333336, -46.5, -51.0, -54.333333333333336, -55.0, -59.4, 0]
[-42.333333333333336, -45.8, -51.2, -53.75, -55.0, -57.4, 0]
[-44.0, -46.0, -51.4, -53.75, -55.2, -57.0, 0]
[-44.0, -46.8, -51.0, -53.75, -55.2,

In [56]:
## this is for new_loc_1-3 folders

numLoc = 3   ## number of locations
numVec = 50 ## number of vecters per loc
numScan = 9  ## number scans per vector folder

vectors = []
## for each location file i.e. range(1,numLoc+1)
for i in range(1,numLoc+1):
    locPath = "new_loc_" + str(i) + "/"
    db = 1   ## db files are arranged from 1 - 450 in this raw data set
             ## so we will just count from 1 thru 450
    ## for each vec file i.e. range(1,numVec+1)
    for j in range(1,numVec+1):
        vecPath = "vec_" + str(j) + "/"
        minRan = db
        maxRan = db+numScan
        
        ## handles portion of dbXX.csv in the vec
        for k in range(minRan,maxRan):
            ## print(db)
            dbPath = "db%s.csv" % k
            db+=1
            path = locPath + vecPath + dbPath
            ## "file_{}.dat".format(i)
            data = pd.read_csv(path)
        
            new = data[['MAC Address','RSSI']]
            new = new.set_index('MAC Address')
            if j == 1:
                if k == 1:
                    
                    ## using top 8 initial signal strengths (can be changed for specific case)
                    ## these 8 will refer to 8 highest potential APs
                    
                    ref = new[0:8]
                    reflist = ref.index.values.tolist()
                    # pickle the list for creating prdictions vectors later
                    with open('reflist.pkl', 'wb') as f:
                        pickle.dump(reflist, f)
                    ## print(reflist)
                    base = pd.DataFrame(index=ref.index)
                    base.to_pickle("refbase.pkl")
            vecdf = base
            new = new['RSSI'].to_dict()
            applist = []
            for h in reflist:
                toapp = 0
                for x,y in new.items():
                    if x == h:
                        toapp = y
                applist.append(toapp)
            add = np.asarray(applist)
            vecdf[('RSSI%d' % (db - minRan))] = add
        ## vecdf now refers to all rssi values sorted by scan and AP(MAC Address) for 1 vector
        ## We must now transpose each df, find the average RSSI and append the resulting vector
        ## to our final data df, then export as a csv
        vecdf = vecdf.T
        ## replace all zero values with nan
        vecdf.replace(0, np.nan, inplace= True)
        ## print(vecdf.head())
        ## create the vector which will be fed to NN
        vec = []
    
        for column in vecdf:
            avg = vecdf[column].mean()
            vec.append(avg)
        vec.append(i-1)
        vectors.append(vec)
        print(vec)
print(vectors)
col = reflist
col.append("loc")
allvec=pd.DataFrame(vectors,columns=col)
print(allvec)
allvec.to_csv('new_trainingdata.csv', index=False)

[-46.285714285714285, -45.125, -47.142857142857146, -49.285714285714285, -55.5, -54.857142857142854, -57.75, -59.125, 0]
[-46.0, -45.888888888888886, -46.55555555555556, -51.0, -56.375, -55.44444444444444, -58.111111111111114, -57.888888888888886, 0]
[-45.888888888888886, -46.375, -49.55555555555556, -49.625, -56.22222222222222, -54.888888888888886, -55.77777777777778, -58.77777777777778, 0]
[-45.2, -49.2, -55.666666666666664, -55.333333333333336, -54.0, -55.0, -51.666666666666664, -62.22222222222222, 0]
[-46.0, -49.0, -57.0, -55.333333333333336, -53.25, -54.111111111111114, -52.111111111111114, -62.22222222222222, 0]
[-46.333333333333336, -49.5, -56.75, -56.75, -53.5, -55.44444444444444, -51.44444444444444, -61.44444444444444, 0]
[-46.55555555555556, -49.0, -55.42857142857143, -55.2, -53.666666666666664, -54.666666666666664, -52.55555555555556, -61.888888888888886, 0]
[-45.55555555555556, -48.0, -56.666666666666664, -56.5, -53.888888888888886, -55.666666666666664, -53.111111111111114,